# Linear Regression

## Initialization of libraries and Dataset

In [5]:
%matplotlib inline
import os
import sys
import matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from datetime import datetime, date
from sklearn.metrics import precision_recall_curve, average_precision_score

from statsmodels.formula.api import ols
import statsmodels.api as sm

matplotlib.style.use('ggplot')

In [9]:
# Downloading and preprocessing the dataset
# The variable "tab" has integers datetime
execfile('Initialization.py')
X.head()

,sender,receiver,datetime
0,1,2,733774
1,1,3,733774
2,1,4,733774
3,1,5,733774
4,1,6,733774


## Linear Regression

### Dataset Temporal Split
We use (sender,receiver)_t matrix for t between t_0 and t_1. This temporal window shifts as t_0 incresases.  
t_0 is s.t. t_0+t_1 <= T.shape[2] = 238  
  
Since we would like to move this window around 3 weeks, we choose t_0 <= 21 and a constant window size of length 238-21=217

In [10]:
t_0 = 7
t_1 = t_0 + 217

if t_1 <= T.shape[2]:
    T_train = T[:, :, t_0:t_1]
    T_test = T[:, :, t_1:]
    print('Initial time: t_0 = %d [%s / %d]\nFinal time:   t_1 = %d [%s / %d]') %(t_0,
                                                                                  date.fromordinal(k_list[t_0]),
                                                                                  k_list[t_0],
                                                                                  t_1,
                                                                                  date.fromordinal(k_list[t_1]),
                                                                                  k_list[t_1])
else:
    print('t_0=%d is superior to the maximal value 21!') %(t_0)

Initial time: t_0 = 7 [2010-01-10 / 733782]
Final time:   t_1 = 224 [2010-09-16 / 734031]


In [11]:
#Proportion of non-zero entries of the tensor
mail_rate = np.count_nonzero(T_train) / float(T_train.shape[0] * T_train.shape[1] * T_train.shape[2])

In [51]:
from scipy import linalg

X2 = tab[[0,1]]
n = X2.size

In [52]:
import datetime

dates = tab.datetime.dt.date[:]
dates = dates.apply(lambda dates: dates.toordinal())

In [53]:
dates = pd.Series(dates, name='date')
X2 = pd.concat([X2,dates], axis = 1)

In [54]:
# print(tab.datetime.dt.date[0], tab.datetime.dt.date[100])
print(tab.datetime.dt.date[100].toordinal(), tab.datetime.dt.date[0].toordinal())

(733776, 733774)


### Dataset splitting into train set and test set

We choose to spilt the data set temporally such that the train set contains 75% of the emails.

In [55]:
X2.sort_values(['date', 'sender', 'receiver'], ascending = [1, 1, 1], inplace=True)

train_split = 0.75
train_index = int(round(train_split*X2.shape[0]))

In [56]:
XTrain2 = X2[:train_index]

XTest2 = X2[train_index:]

In [177]:
XTest2.date.iloc[-1] - XTrain2.date.iloc[train_index - 1]

66

In [58]:
reg = ols('receiver ~ sender + date', data = XTrain2)

In [ ]:
print resultats.iloc[:,0:6]

In [226]:
a = []
a[0] = 1

IndexError: list assignment index out of range

In [227]:
# Another way to collect the data

senders = pd.unique(XTrain2.sender)

maxi = max(XTest2.date)
datum = []

for current_sender, current_receiver in pd.unique(zip(XTrain2.sender, XTrain2.receiver)):
    current_X = XTrain2[(XTrain2.sender == current_sender)&(XTrain2.receiver == current_receiver)] 
    datum = datum + [maxi-max(current_X.date)]


In [229]:
max(datum)

269

In [300]:
# Another way to collect the data

senders = pd.unique(XTrain2.sender)

emails = {}
results = {}

for current_sender, current_receiver in pd.unique(zip(XTrain2.sender, XTrain2.receiver)):
    current_X = XTrain2[(XTrain2.sender == current_sender)&(XTrain2.receiver == current_receiver)] 
    
    if len(current_X.date) >= 2:
        Y = current_X
        # Add the index as a variable
        Y.index = range(len(Y))
        Y = Y.reset_index(level = 0, inplace = False) 
        
        # Linear regression
        reg = ols('date ~ index', data = Y)
        res = reg.fit()
        
        # Test set
        pred_X = XTest2[(XTest2.sender == current_sender)&(XTest2.receiver == current_receiver)] 
        
        # Prediction 
        index = pd.Series(range(len(Y), len(Y) + 300), name='index')
        Xtest = sm.add_constant(index)
        pred_receiver = reg.predict(res.params, Xtest)
        
        #for day in pred_receiver:
            #print datetime.fromordinal(int(round(day))).date()

        # For displaying, use pd.unique for test set
        pred_X = pd.Series(pd.unique(pred_X.date), name = 'date')
        
        #resultats = pd.concat([results, pd.Series(['NaN']*len(pred_X), name = (current_sender,current_receiver)), pred_X['date'], pd.Series(pred_receiver.round(), name = ('prediction ', (current_sender,current_receiver)))[:len(pred_X)]], axis = 1)
        pred_receiver = pred_receiver.round()
        pred_receiver = pred_receiver[pred_receiver <= maxi]
        for j in range(len(pred_receiver)):
            pred_receiver[j] = int(pred_receiver[j])
            
        results[(current_sender,current_receiver)] = pd.Series(pred_receiver)
        
        emails[(current_sender,current_receiver)] = pred_X
  

In [301]:
# emails

In [304]:
len(results)

3339

In [305]:
counts = {key: {'tp':0, 'nb_mails_test':0, 'nb_mails_pred':0} for key in results.keys()}
#counts

In [306]:
for keys in results.keys():
    tp_count = 0
    for val in results[keys]:
        if val in emails[keys]:
            tp_count = tp_count + 1
            
    counts[keys]['tp'] = count
    counts[keys]['nb_mails_pred'] = len(results[keys])
    counts[keys]['nb_mails_test'] = len(emails[keys])
